In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
df = pd.read_csv("HillaryEmails.csv")
df = df[['Id','ExtractedBodyText']].dropna()

In [3]:
def clean_email_text(text):
    text = text.replace('\n'," ")
    text = re.sub(r"-", " ", text) 
    text = re.sub(r"\d+/\d+/\d+", "", text)
    text = re.sub(r"[0-2]?[0-9]:[0-6][0-9]", "", text)
    text = re.sub(r"[\w]+@[\.\w]+", "", text) #邮件地址，没意义
    text = re.sub(r"/[a-zA-Z]*[:\//\]*[A-Za-z0-9\-_]+\.+[A-Za-z0-9\.\/%&=\?\-_]+/i", "", text) #网址，没意义
    pure_text = ''
    # 以防还有其他特殊字符（数字）等等，我们直接把他们loop一遍，过滤掉
    for letter in text:
        # 只留下字母和空格
        if letter.isalpha() or letter==' ':
            pure_text += letter
    # 再把那些去除特殊字符后落单的单词，直接排除。
    # 我们就只剩下有意义的单词了。
    text = ' '.join(word for word in pure_text.split() if len(word)>1)
    return text

In [4]:
docs = df['ExtractedBodyText']
docs = docs.apply(lambda s: clean_email_text(s))  

In [39]:
docs[1]

'Thursday March PM Latest How Syria is aiding Qaddafi and more Sid hrc memo syria aiding libya docx hrc memo syria aiding libya docx March For Hillary'

In [15]:
doclist = docs.values

In [16]:
doclist[0]

'Thursday March PM Latest How Syria is aiding Qaddafi and more Sid hrc memo syria aiding libya docx hrc memo syria aiding libya docx March For Hillary'

In [17]:
doclist.shape

(6742,)

In [18]:
from gensim import corpora, models, similarities
import gensim

In [19]:
stoplist = ['very', 'ourselves', 'am', 'doesn', 'through', 'me', 'against', 'up', 'just', 'her', 'ours', 
            'couldn', 'because', 'is', 'isn', 'it', 'only', 'in', 'such', 'too', 'mustn', 'under', 'their', 
            'if', 'to', 'my', 'himself', 'after', 'why', 'while', 'can', 'each', 'itself', 'his', 'all', 'once', 
            'herself', 'more', 'our', 'they', 'hasn', 'on', 'ma', 'them', 'its', 'where', 'did', 'll', 'you', 
            'didn', 'nor', 'as', 'now', 'before', 'those', 'yours', 'from', 'who', 'was', 'm', 'been', 'will', 
            'into', 'same', 'how', 'some', 'of', 'out', 'with', 's', 'being', 't', 'mightn', 'she', 'again', 'be', 
            'by', 'shan', 'have', 'yourselves', 'needn', 'and', 'are', 'o', 'these', 'further', 'most', 'yourself', 
            'having', 'aren', 'here', 'he', 'were', 'but', 'this', 'myself', 'own', 'we', 'so', 'i', 'does', 'both', 
            'when', 'between', 'd', 'had', 'the', 'y', 'has', 'down', 'off', 'than', 'haven', 'whom', 'wouldn', 
            'should', 've', 'over', 'themselves', 'few', 'then', 'hadn', 'what', 'until', 'won', 'no', 'about', 
            'any', 'that', 'for', 'shouldn', 'don', 'do', 'there', 'doing', 'an', 'or', 'ain', 'hers', 'wasn', 
            'weren', 'above', 'a', 'at', 'your', 'theirs', 'below', 'other', 'not', 're', 'him', 'during', 'which']

In [20]:
texts = [[word for word in doc.lower().split() if word not in stoplist] for doc in doclist]

In [24]:
texts[0]

['thursday',
 'march',
 'pm',
 'latest',
 'syria',
 'aiding',
 'qaddafi',
 'sid',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 'docx',
 'hrc',
 'memo',
 'syria',
 'aiding',
 'libya',
 'docx',
 'march',
 'hillary']

# building corpus

In [25]:
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

In [31]:
corpus[13]

[(51, 1), (505, 1), (506, 1), (507, 1), (508, 1)]

No14 Email has 6 meaningful words. no 50 word appear once.

# building model

In [32]:
lda = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=20)

In [44]:
lda.print_topic(10, topn=5)

'0.040*"call" + 0.012*"pm" + 0.011*"talk" + 0.011*"sunday" + 0.008*"ops"'

In [38]:
lda.print_topics(num_topics=20, num_words=5)

[(0,
  '0.011*"good" + 0.010*"miliband" + 0.009*"great" + 0.007*"see" + 0.007*"sounds"'),
 (1,
  '0.031*"settlements" + 0.027*"israeli" + 0.023*"israel" + 0.015*"settlement" + 0.012*"boehner"'),
 (2,
  '0.024*"pm" + 0.013*"office" + 0.010*"message" + 0.009*"sullivan" + 0.009*"huma"'),
 (3,
  '0.022*"pls" + 0.019*"print" + 0.008*"north" + 0.008*"missile" + 0.007*"waldorf"'),
 (4,
  '0.008*"senate" + 0.007*"us" + 0.006*"said" + 0.006*"state" + 0.006*"taliban"'),
 (5,
  '0.108*"ok" + 0.010*"discuss" + 0.009*"cr" + 0.007*"isabelle" + 0.006*"lets"'),
 (6,
  '0.015*"pm" + 0.008*"roger" + 0.007*"blackberry" + 0.007*"via" + 0.006*"claims"'),
 (7,
  '0.032*"thx" + 0.030*"pis" + 0.015*"doc" + 0.014*"happy" + 0.013*"print"'),
 (8,
  '0.012*"yes" + 0.011*"percent" + 0.010*"would" + 0.008*"democrats" + 0.007*"republican"'),
 (9,
  '0.113*"pm" + 0.049*"office" + 0.044*"secretarys" + 0.029*"meeting" + 0.028*"room"'),
 (10,
  '0.040*"call" + 0.012*"pm" + 0.011*"talk" + 0.011*"sunday" + 0.008*"ops"'),
